In [ ]:
from pathlib import Path

from rosbags.highlevel import AnyReader
from rosbags.typesys import Stores, get_typestore
import matplotlib.pyplot as plt 
import numpy as np
from scipy.optimize import minimize
from arduino_pid import ArduinoPID 

In [ ]:
ROOT = Path.cwd().parent  # motor_identification/parent -> root projektu
MCAP = ROOT / "data" / "ident_data_1_forward_drive" / "ident_data_1_forward_drive.mcap"
bagpath = Path(MCAP)
typestore = get_typestore(Stores.ROS2_JAZZY)
# Create reader instance and open for reading.
topic_list = [
    "/wheel_velocity_ref",
    "/joint_states",
    "/odom",
    "/imu/out"
]

dict_msg = {topic: [] for topic in topic_list}

with AnyReader([bagpath], default_typestore=typestore) as reader:
    for topic in topic_list:
        connections = [x for x in reader.connections if x.topic == topic]
        for connection, timestamp, rawdata in reader.messages(connections=connections):
            msg = reader.deserialize(rawdata, connection.msgtype)
            dict_msg[topic].append(msg)

In [ ]:
vel_ref  = np.array([msg.data for msg in dict_msg["/wheel_velocity_ref"]])
vel_act  = np.array([msg.velocity for msg in dict_msg["/joint_states"]])
vel_ref_right = vel_ref[:, 0]
vel_ref_left  = vel_ref[:, 1] 
vel_act_right = vel_act[:, 0]
vel_act_left  = vel_act[:, 1]
pass

In [ ]:
fig,ax = plt.subplots(2,1, figsize=(10,6), sharex=True)

ax[0].plot(vel_ref[:,0], label="right wheel ref")
ax[0].plot(vel_ref[:,1], label="left wheel ref")
ax[0].set_ylabel("Wheel velocity [rad/s]")
ax[0].set_xlabel("Sample index")
ax[0].legend()

ax[1].plot(vel_act[:,0], label="right wheel act")
ax[1].plot(vel_act[:,1], label="left wheel act")
ax[1].set_ylabel("Wheel velocity [rad/s]")
ax[1].set_xlabel("Sample index")
ax[1].legend()

plt.show()

##### Identification


In [ ]:
def motor_step(omega, u_volt, K, tau, Ts):
    return omega + Ts * (-omega / tau + K / tau * u_volt)

def simulate_closed_loop(params, vel_ref, Ts):
    K, tau = params
    pid = ArduinoPID(Kp=40, Ki=30, Kd=0.1, Ts=Ts)
    omega = 0.0
    omega_hist = []
    for ref in vel_ref:
        u_pwm = pid.step(ref, omega)
        u_volt = 12.0 / 1023.0 * u_pwm
        omega = motor_step(omega, u_volt, K, tau, Ts)
        omega_hist.append(omega)
    return np.array(omega_hist)

def cost_function(params, vel_ref, vel_meas, Ts):
    K, tau = params
    if K <= 0 or tau <= 0:
        return 1e9
    vel_sim = simulate_closed_loop(params, vel_ref, Ts)
    return np.mean((vel_sim - vel_meas)**2)

In [ ]:
N = min(len(vel_ref_right), len(vel_act_right))

vel_ref_right = vel_ref_right[:N]
vel_act_right = vel_act_right[:N]

In [ ]:
fig,ax = plt.subplots(2,1, figsize=(10,6), sharex=True)

ax[0].plot(vel_ref_right, label="right wheel ref")
ax[0].set_ylabel("Wheel velocity [rad/s]")
ax[0].set_xlabel("Sample index")
ax[0].legend()

ax[1].plot(vel_act_right, label="right wheel act")
ax[1].set_ylabel("Wheel velocity [rad/s]")
ax[1].set_xlabel("Sample index")
ax[1].legend()

plt.show()

In [ ]:
initial_guess = [1.0, 0.1]  # K [rad/s/V], tau [s]
Ts = 0.01  # 10 ms

result = minimize(
    cost_function,
    initial_guess,
    args=(vel_ref_right, vel_act_right, Ts),
    method="Nelder-Mead"
)

K_id, tau_id = result.x
print("Identified parameters:")
print("K =", K_id)
print("tau =", tau_id)

In [ ]:
vel_sim = simulate_closed_loop([K_id, tau_id], vel_ref_right, Ts)

plt.figure(figsize=(10,5))
plt.plot(vel_act_right, label="measured")
plt.plot(vel_sim, label="simulated")
plt.plot(vel_ref_right, "--", label="reference", alpha=0.5)
plt.legend()
plt.grid()
plt.show()

In [ ]:
def simulate_closed_loop_pid_opt(params_pid, vel_ref, K, tau, Ts):
    Kp, Ki, Kd = params_pid
    pid = ArduinoPID(Kp, Ki, Kd, Ts)
    omega = 0.0
    omega_hist = []
    for ref in vel_ref:
        u_pwm = pid.step(ref, omega)
        u_volt = 12.0 / 1023.0 * u_pwm
        omega = motor_step(omega, u_volt, K, tau, Ts)
        omega_hist.append(omega)
    return np.array(omega_hist)

def cost_pid_opt(params_pid, vel_ref, vel_meas, K, tau, Ts):
    Kp, Ki, Kd = params_pid
    if any(p < 0 for p in [Kp, Ki, Kd]):
        return 1e9  # zabezpieczenie przed ujemnymi wartościami
    vel_sim = simulate_closed_loop_pid_opt(params_pid, vel_ref, K, tau, Ts)
    return np.sqrt(np.mean((vel_sim - vel_meas)**2))

In [ ]:
from scipy.optimize import minimize

Ts = 0.01  # Twój krok dyskretny
vel_ref = vel_ref_right
vel_meas = vel_act_right  # po interpolacji

# Początkowy PID (IMC / poprzednie guess)
init_pid = [2.2, 6.1, 0.0]

res = minimize(
    cost_pid_opt,
    x0=init_pid,
    args=(vel_ref, vel_meas, 2.27, 0.358, Ts),
    method='Nelder-Mead',
    options={'maxiter':500, 'disp': True}
)

Kp_opt, Ki_opt, Kd_opt = res.x
print("Optimal PID:", Kp_opt, Ki_opt, Kd_opt)

In [ ]:
vel_sim_opt = simulate_closed_loop_pid_opt([Kp_opt, Ki_opt, Kd_opt], vel_ref, 2.27, 0.358, Ts)

import matplotlib.pyplot as plt
plt.figure(figsize=(10,4))
plt.plot(vel_ref, '--', label='vel_ref')
plt.plot(vel_meas, '.', alpha=0.3, label='vel_act')
plt.plot(vel_sim_opt, '-', label='vel_sim_opt')
plt.legend()
plt.grid()
plt.show()
